In [2]:
from modnet.preprocessing import MODData
data=MODData.load('DATAFILES/matbench_perovskites_moddata.pkl.gz')

2022-12-10 21:07:00,121 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f4fe8360b50> object, created with modnet version 0.1.12


In [4]:
data.df_structure

structure
id                                                        
id0      [[0. 0. 0.] Rh, [1.97726555 1.97726555 1.97726...
id1      [[2.54041798 0.         0.        ] Hf, [1.020...
id2      [[0.60790913 0.         0.        ] Re, [2.186...
id3      [[2.83091357 0.         0.        ] W, [2.6573...
id4      [[0.00518937 0.         0.        ] Bi, [2.172...
...                                                    ...
id18923  [[4.44077598 0.         0.        ] Rb, [2.652...
id18924  [[4.56913824e-03 7.21569024e-19 0.00000000e+00...
id18925  [[0.0040044 0.        0.       ] Zn, [1.821570...
id18926  [[0. 0. 0.] Ca, [2.16744896 2.16744896 2.16744...
id18927  [[1.23999712 4.09195837 4.09195837] Al, [2.500...

[18928 rows x 1 columns]

In [ ]:
from matminer.featurizers.structure import OrbitalFieldMatrix
import pickle
featurizers=[OrbitalFieldMatrix(period_tag=False)]
for featurizer in featurizers:
    for idx in list(range(1000,17000,1000)):
        featurizer=OrbitalFieldMatrix()
        featurizer.set_n_jobs(8)
        #data.df_structure=data.df_structure #.sample(n=10, random_state=1)
        print(idx,idx+1000)
        df_feat=featurizer.featurize_dataframe(data.df_structure[idx:idx+1000], 'structure',)
                                               #ignore_errors=True,return_errors=True)
        # save complete featurized dataframe
        pickle.dump(df_feat, open(f"OFM_customfeaturized{idx+1000}.pkl","wb"))

    df_feat=featurizer.featurize_dataframe(data.df_structure[17000:18000], 'structure',)
    # save complete featurized dataframe
    pickle.dump(df_feat, open(f"OFM_customfeaturized18000.pkl","wb"))
    df_feat=featurizer.featurize_dataframe(data.df_structure[18000:], 'structure',)
    # save complete featurized dataframe
    pickle.dump(df_feat, open(f"OFM_customfeaturized18928.pkl","wb"))

In [ ]:
import pickle, glob
OFMs=[]
for file in glob.glob("OFM_*"):
    OFM=pickle.load(open(file,"rb"))
    OFMs.append(OFM)
    print(f"Loaded {file}.")
import pandas as pd
featurizedOFM=pd.concat(OFMs,axis=0)
featurizedOFM

### Now to featurize with SOAP. I will try to make the columns a bit more meaningful.

In [ ]:
import numpy as np
from dscribe.descriptors import SOAP
import pickle
import pymatgen
from pymatgen.io.ase import AseAtomsAdaptor
from ase.data import atomic_numbers
import os
import pandas as pd
structures=data.df_structure['structure']
try:
    structures_ase=pickle.load(open("ase_structures.pkl","rb"))
except:
    structures_ase=list(map(AseAtomsAdaptor.get_atoms,structures))
    pickle.dump(structures_ase,open("ase_structures.pkl","wb"))
    
## declaring the SOAP featurizer
species=list(atomic_numbers.keys())[1:] ## all chemical species
nmax=8
lmax=6
rcut=5
average_soap = SOAP(species=species,
rcut=rcut, nmax=nmax, lmax=lmax,
    average="inner",
    crossover=True,
    periodic=True,
    sparse=False
)
ncpus=os.cpu_count()
## this is very memory intensive has to be splitted
slices=[None]+list(range(100,len(structures_ase),100))+[None]
for i, slice1 in list(enumerate(slices))[:-1]:
    results = average_soap.create(structures_ase[slice1:slices[i+1]], n_jobs=ncpus)
    pickle.dump(results, open(f"SOAP_perovsk_featurized_{i}.pkl","wb"))
    print(f"{i} out of {len(slices)-2} subsets to complete SOAP featurization")

In [23]:
slices=[None]+list(range(100,18928,100))+[None]
import pickle
structures_ase=pickle.load(open("ase_structures.pkl","rb"))
for i, slice1 in list(enumerate(slices))[:-1]:
    print(slice1,slices[i+1])
    print(structures_ase[slice1:slices[i+1]])
    print("")

None 100
[Atoms(symbols='RhTeN3', pbc=True, cell=[3.9545311067746507, 3.9545311067746507, 3.9545311067746507]), Atoms(symbols='HfTeO3', pbc=True, cell=[4.2894318978043495, 4.2894318978043495, 4.2894318978043495]), Atoms(symbols='ReAsFO2', pbc=True, cell=[4.292638763849676, 4.292638763849676, 4.292638763849676]), Atoms(symbols='WReSO2', pbc=True, cell=[4.183730564596991, 4.183730564596991, 4.183730564596991]), Atoms(symbols='BiHfFO2', pbc=True, cell=[4.281144253928289, 4.281144253928289, 4.281144253928289]), Atoms(symbols='YLiFNO', pbc=True, cell=[3.781460058732497, 3.781460058732497, 3.781460058732497]), Atoms(symbols='BeWFO2', pbc=True, cell=[4.0728405739029645, 4.0728405739029645, 4.0728405739029645]), Atoms(symbols='ScAsNO2', pbc=True, cell=[3.832320939768715, 3.832320939768715, 3.832320939768715]), Atoms(symbols='CrTeN2O', pbc=True, cell=[3.9705455979471473, 3.9705455979471473, 3.9705455979471473]), Atoms(symbols='ScRbSO2', pbc=True, cell=[4.816532949685957, 4.816532949685957, 4.81

[Atoms(symbols='RbTaO3', pbc=True, cell=[4.115875418170205, 4.115875418170205, 4.115875418170205]), Atoms(symbols='BLaO3', pbc=True, cell=[4.651974580120703, 4.651974580120703, 4.651974580120703]), Atoms(symbols='GePtNO2', pbc=True, cell=[4.005192148741399, 4.005192148741399, 4.005192148741399]), Atoms(symbols='MoTaO3', pbc=True, cell=[4.057675635925539, 4.057675635925539, 4.057675635925539]), Atoms(symbols='YPbN3', pbc=True, cell=[4.255361286373359, 4.255361286373359, 4.255361286373359]), Atoms(symbols='KSbFNO', pbc=True, cell=[4.287267969078341, 4.287267969078341, 4.287267969078341]), Atoms(symbols='PtRhSO2', pbc=True, cell=[4.340600914756946, 4.340600914756946, 4.340600914756946]), Atoms(symbols='AgRuN3', pbc=True, cell=[4.008395405702043, 4.008395405702043, 4.008395405702043]), Atoms(symbols='TiPdO3', pbc=True, cell=[4.013065794349127, 4.013065794349127, 4.013065794349127]), Atoms(symbols='Nb2SO2', pbc=True, cell=[4.400170705050235, 4.400170705050235, 4.400170705050235]), Atoms(sym

[Atoms(symbols='BFeN3', pbc=True, cell=[3.6000599406485576, 3.6000599406485576, 3.6000599406485576]), Atoms(symbols='HgBiO3', pbc=True, cell=[4.309844870611419, 4.309844870611419, 4.309844870611419]), Atoms(symbols='InGaN2O', pbc=True, cell=[3.9531822952875384, 3.9531822952875384, 3.9531822952875384]), Atoms(symbols='InZrNO2', pbc=True, cell=[4.249280264875794, 4.249280264875794, 4.249280264875794]), Atoms(symbols='GaFeFNO', pbc=True, cell=[3.8840807563850612, 3.8840807563850612, 3.8840807563850612]), Atoms(symbols='LaGaFO2', pbc=True, cell=[4.136375557525437, 4.136375557525437, 4.136375557525437]), Atoms(symbols='YAgN2O', pbc=True, cell=[4.103911102322338, 4.103911102322338, 4.103911102322338]), Atoms(symbols='BiSiFNO', pbc=True, cell=[3.9883759388878195, 3.9883759388878195, 3.9883759388878195]), Atoms(symbols='AuBiN2O', pbc=True, cell=[4.324418059889259, 4.324418059889259, 4.324418059889259]), Atoms(symbols='CsRhFO2', pbc=True, cell=[4.313333946891767, 4.313333946891767, 4.3133339468

[Atoms(symbols='NiSnFO2', pbc=True, cell=[4.146397061991604, 4.146397061991604, 4.146397061991604]), Atoms(symbols='TiBiN3', pbc=True, cell=[4.253649319619788, 4.253649319619788, 4.253649319619788]), Atoms(symbols='VCaFO2', pbc=True, cell=[4.320803444127378, 4.320803444127378, 4.320803444127378]), Atoms(symbols='GeCsSO2', pbc=True, cell=[4.95240659181098, 4.95240659181098, 4.95240659181098]), Atoms(symbols='MgCdNO2', pbc=True, cell=[4.175844306912015, 4.175844306912015, 4.175844306912015]), Atoms(symbols='RhSnFO2', pbc=True, cell=[4.181057486534517, 4.181057486534517, 4.181057486534517]), Atoms(symbols='KZrSO2', pbc=True, cell=[4.66996315435506, 4.66996315435506, 4.66996315435506]), Atoms(symbols='RbBaO3', pbc=True, cell=[4.610429583880766, 4.610429583880766, 4.610429583880766]), Atoms(symbols='MoBaO3', pbc=True, cell=[4.613078670151452, 4.613078670151452, 4.613078670151452]), Atoms(symbols='GePbNO2', pbc=True, cell=[4.326945401610301, 4.326945401610301, 4.326945401610301]), Atoms(symb

[Atoms(symbols='GaSbO3', pbc=True, cell=[3.986990587640283, 3.986990587640283, 3.986990587640283]), Atoms(symbols='W2O3', pbc=True, cell=[4.009271406655795, 4.009271406655795, 4.009271406655795]), Atoms(symbols='CdScFO2', pbc=True, cell=[4.1522480827373, 4.1522480827373, 4.1522480827373]), Atoms(symbols='AlBiFO2', pbc=True, cell=[4.497473647361479, 4.497473647361479, 4.497473647361479]), Atoms(symbols='NaLaN2O', pbc=True, cell=[4.591894558350153, 4.591894558350153, 4.591894558350153]), Atoms(symbols='HfTlN3', pbc=True, cell=[4.107717996512127, 4.107717996512127, 4.107717996512127]), Atoms(symbols='RhTlO3', pbc=True, cell=[4.274536217956951, 4.274536217956951, 4.274536217956951]), Atoms(symbols='PbRbO3', pbc=True, cell=[4.643734318538381, 4.643734318538381, 4.643734318538381]), Atoms(symbols='AlNbFNO', pbc=True, cell=[4.172025715843324, 4.172025715843324, 4.172025715843324]), Atoms(symbols='BaSiFO2', pbc=True, cell=[4.205316337653879, 4.205316337653879, 4.205316337653879]), Atoms(symbol

[Atoms(symbols='SrWN3', pbc=True, cell=[4.077485833104291, 4.077485833104291, 4.077485833104291]), Atoms(symbols='AsCuSO2', pbc=True, cell=[4.1774710640907236, 4.1774710640907236, 4.1774710640907236]), Atoms(symbols='ScBFO2', pbc=True, cell=[3.6548177850458115, 3.6548177850458115, 3.6548177850458115]), Atoms(symbols='VIrSO2', pbc=True, cell=[4.29811013684419, 4.29811013684419, 4.29811013684419]), Atoms(symbols='ZnNiN2O', pbc=True, cell=[3.754843417478881, 3.754843417478881, 3.754843417478881]), Atoms(symbols='AsPdNO2', pbc=True, cell=[3.9826156696734634, 3.9826156696734634, 3.9826156696734634]), Atoms(symbols='AsReSO2', pbc=True, cell=[4.229139393951406, 4.229139393951406, 4.229139393951406]), Atoms(symbols='ZrIrO3', pbc=True, cell=[4.05466444567771, 4.05466444567771, 4.05466444567771]), Atoms(symbols='GeOsN3', pbc=True, cell=[3.9709630005828678, 3.9709630005828678, 3.9709630005828678]), Atoms(symbols='NaBaFO2', pbc=True, cell=[4.966795242966614, 4.966795242966614, 4.966795242966614]),

[Atoms(symbols='TlBiO3', pbc=True, cell=[4.349247310936832, 4.349247310936832, 4.349247310936832]), Atoms(symbols='NbBiN3', pbc=True, cell=[4.2395741265414, 4.2395741265414, 4.2395741265414]), Atoms(symbols='WZnO3', pbc=True, cell=[3.910013847823804, 3.910013847823804, 3.910013847823804]), Atoms(symbols='NiYN3', pbc=True, cell=[4.420239428895007, 4.420239428895007, 4.420239428895007]), Atoms(symbols='NaCsN2O', pbc=True, cell=[4.523403130423973, 4.523403130423973, 4.523403130423973]), Atoms(symbols='PtReFNO', pbc=True, cell=[3.97355326557604, 3.97355326557604, 3.97355326557604]), Atoms(symbols='LaSrO3', pbc=True, cell=[4.583778503260977, 4.583778503260977, 4.583778503260977]), Atoms(symbols='LiAuSO2', pbc=True, cell=[4.281212312302197, 4.281212312302197, 4.281212312302197]), Atoms(symbols='KGaO3', pbc=True, cell=[4.035486643360293, 4.035486643360293, 4.035486643360293]), Atoms(symbols='NiMoNO2', pbc=True, cell=[3.9217977172165197, 3.9217977172165197, 3.9217977172165197]), Atoms(symbols=

In [27]:
len(structures_ase)

18928

In [12]:
slices[i]

18900

In [19]:
# #df_feat=featurizer.featurize_dataframe(data.df_structure.iloc[[5530]], 'structure')
# #df_feat, 
# def nspec(x):
#     return len(x.species)
# data.df_structure['species']= data.df_structure['structure'].apply(nspec) #.iloc[5530].values[0].species
# data.df_structure['species'].sort_values()

In [20]:
# import pandas as pd
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# data.df_structure.iloc[5529]['structure']


Structure Summary
Lattice
    abc : 3.930170626613603 3.930170626613603 3.930170626613603
 angles : 90.0 90.0 90.0
 volume : 60.706363276205764
      A : 3.930170626613603 0.0 0.0
      B : 0.0 3.930170626613603 0.0
      C : 0.0 0.0 3.930170626613603
PeriodicSite: Ta (2.6090, 3.9302, 0.0000) [0.6638, 1.0000, 0.0000]
PeriodicSite: Ga (0.0259, 1.9651, 1.9651) [0.0066, 0.5000, 0.5000]
PeriodicSite: N (3.5263, 3.9302, 1.9651) [0.8972, 1.0000, 0.5000]
PeriodicSite: N (3.5263, 1.9651, 3.9302) [0.8972, 0.5000, 1.0000]
PeriodicSite: N (1.9054, 1.9651, 1.9651) [0.4848, 0.5000, 0.5000]